# Question 1

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np

%matplotlib inline
import os
import pandas as pd
import numpy as np
import xgboost
import matplotlib.pyplot as plt
import scikitplot as sckplt
import time
import re
import scipy.stats as stat
from math import sqrt
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn import linear_model, tree, neighbors, svm, ensemble

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
# setting OS directory
os.chdir('C:\\Users\\rckar\\OneDrive\\Documents\\MSBA\\Fall Semester\\6420 Predictive Analytics\\HW3')

In [3]:
# Reading Data
df = pd.read_excel("HW3.xlsx")
df.head(2)
# df.dtypes

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.0
1,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0.0


## Part A

In [4]:
# Data pre-processing

# checking for null values
df.isnull().values.any()

False

In [5]:
# Selecting required columns
X_df = df.iloc[:,1:23]
y_df = df.iloc[:,24:]

##### Creating the Train and Test split

In [6]:
X_train, X_test_holdout, y_train, y_test_holdout = train_test_split(X_df, y_df, test_size=0.25, random_state=42)

##### Normalizing data

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

x_test_scaled = scaler.fit_transform(X_test_holdout)
X_test_holdout = pd.DataFrame(x_test_scaled)

### Linear Regression

In [8]:
start = time.process_time()
# create linear regression object 
lr = linear_model.LinearRegression()

# cross validation 
scores = cross_val_score(lr, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

print("Below are the mean squared errors for each model run")
print(scores)
print(" ")
print("Mean score: %0.2f " % (abs(scores.mean())))

# Model fit on training data and predicting on testing data
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test_holdout)

# Model performance on testing data
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('Explained Variance:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print(" ")
print('Time Taken = ', time.process_time() - start)

Below are the mean squared errors for each model run
[-26275.29554033 -14376.09803371 -17587.92789557 -20491.5690275
  -8717.39746787]
 
Mean score: 17489.66 
 
 
Model performance on testing data
Mean Absolute Error: 74.91249390506677
Root Mean Squared Error: 116.83025324452498
r2: 0.5681889233817079
 
Time Taken =  0.09375


## Lasso Regression

In [9]:
start = time.process_time()

lasso = linear_model.Lasso()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_lasso = GridSearchCV(lasso, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_lasso.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_lasso.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_lasso.best_params_)

print(" ")
print("Best estimator")
print (grid_lasso.best_estimator_)

# predicting on test data
lasso = grid_lasso.best_estimator_
lasso.fit(X_train,y_train)
y_pred = lasso.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  17392.052253586684
 
Best parameters
{'alpha': 0.30000000000000004}
 
Best estimator
Lasso(alpha=0.30000000000000004, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 73.91843476603411
Root Mean Squared Error: 116.69198704602064
r2: 0.5692103976083474
Time Taken =  1.0625


## Ridge Regression

In [10]:
start = time.process_time()

ridge = linear_model.Ridge()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_ridge = GridSearchCV(ridge, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_ridge.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_ridge.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_ridge.best_params_)

print(" ")
print("Best estimator")
print (grid_ridge.best_estimator_)

# predicting on test data
ridge = grid_ridge.best_estimator_
ridge.fit(X_train,y_train)
y_pred = ridge.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  17488.69461332572
 
Best parameters
{'alpha': 0.1}
 
Best estimator
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
 
 
Model performance on testing data
Mean Absolute Error: 74.7465722508253
Root Mean Squared Error: 116.81233210323924
r2: 0.5683213882796581
Time Taken =  1.125


## KNN

In [11]:
start = time.process_time()

knn = neighbors.KNeighborsRegressor()

#Hyper Parameter tuning
param_set ={'n_neighbors': list(range(1,30)), 'weights': ["uniform", "distance"]}
grid_knn = GridSearchCV(knn, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_knn.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_knn.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_knn.best_params_)

print(" ")
print("Best estimator")
print (grid_knn.best_estimator_)

# predicting on test data
knn = grid_knn.best_estimator_
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  29408.948720197557
 
Best parameters
{'n_neighbors': 6, 'weights': 'distance'}
 
Best estimator
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=6, p=2,
          weights='distance')
 
 
Model performance on testing data
Mean Absolute Error: 87.49654931477475
Root Mean Squared Error: 156.19356346084825
r2: 0.22819176395089769
Time Taken =  20.34375


## Decision Tree

In [12]:
start = time.process_time()

DTree = tree.DecisionTreeRegressor()

#Hyper parameter tuning
param_set ={'max_depth': range(1,20), 'min_samples_split' : range(2,30)}
grid_DTree = GridSearchCV(DTree, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_DTree.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_DTree.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_DTree.best_params_)

print(" ")
print("Best estimator")
print (grid_DTree.best_estimator_)

# predicting on test data
DTree = grid_DTree.best_estimator_
DTree.fit(X_train,y_train)
y_pred = DTree.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  18994.77142166768
 
Best parameters
{'max_depth': 5, 'min_samples_split': 23}
 
Best estimator
DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=23, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
 
 
Model performance on testing data
Mean Absolute Error: 71.19281199747084
Root Mean Squared Error: 126.25980914215464
r2: 0.4956716299205547
Time Taken =  17.71875


## Support Vector Regression

In [13]:
start = time.process_time()

SVR = svm.SVR()
# Hyper parameter tuning using GridSearch
param_set = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1,2,10,100,1000], 'epsilon':[0.05,0.1,0.2,0.3,0.5]},
                    {'kernel': ['linear'], 'C': [1,2,5,10,100], 'epsilon':[0.05,0.1,0.2,0.3,0.5]}]
grid_SVR = GridSearchCV(SVR, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_SVR.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_SVR.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_SVR.best_params_)

print(" ")
print("Best estimator")
print (grid_SVR.best_estimator_)

# predicting on test data
SVR = grid_SVR.best_estimator_
SVR.fit(X_train,y_train)
y_pred = SVR.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  20500.123168359678
 
Best parameters
{'C': 100, 'epsilon': 0.5, 'kernel': 'linear'}
 
Best estimator
SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.5,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
 
 
Model performance on testing data
Mean Absolute Error: 69.88486548166671
Root Mean Squared Error: 129.4836060521223
r2: 0.504335538704678
Time Taken =  48.796875


## Ensemble model : Random Forest

In [14]:
start = time.process_time()
RF = ensemble.RandomForestRegressor()

# Hyper parameter tuning using GridSearch
param_set ={'max_depth': [3,10,20],
            'min_samples_split' :[4,5,10],
            'n_estimators': [100,250,500],
            'bootstrap':[True, False] ,
            'max_features':['auto','sqrt'],
            'n_jobs':[-1]
           }
grid_RF = GridSearchCV(RF, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_RF.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_RF.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_RF.best_params_)

print(" ")
print("Best estimator")
print (grid_RF.best_estimator_)

# predicting on test data
RF = grid_RF.best_estimator_
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print(" ")
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  18302.950738293828
 
Best parameters
{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': -1}
 
Best estimator
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 65.43789920107508
Root Mean Squared Error: 115.2175324613487
r2: 0.5800521264304828
 
Time Taken =  192.671875


## Ensemble model : XGBoost

In [15]:
start = time.process_time()
XGB = xgboost.XGBRegressor(nthreads=-1)

a = stat.beta(10, 1)

# Hyper parameter tuning using GridSearch
param_set ={"n_estimators": stat.randint(3, 40),
            "max_depth": stat.randint(3, 40),
            "learning_rate": stat.uniform(0.05, 0.4),
            "colsample_bytree": a,
            "subsample": a,
            "gamma": stat.uniform(0, 10),
            'n_jobs':[-1]
           }

grid_XGB = RandomizedSearchCV(XGB, param_set, cv=5, scoring='neg_mean_squared_error')
grid_XGB.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_XGB.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_XGB.best_params_)

print(" ")
print("Best estimator")
print (grid_XGB.best_estimator_)

# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
# print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
# print(" ")
print('Time Taken = ', time.process_time() - start)

[16:06:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:06:44] WARNI

Best score achieved across all parameters:  20004.950942708016
 
Best parameters
{'colsample_bytree': 0.9816823845945403, 'gamma': 9.52308795761162, 'learning_rate': 0.16537402362986742, 'max_depth': 9, 'n_estimators': 17, 'n_jobs': -1, 'subsample': 0.9837701891769275}
 
Best estimator
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9816823845945403,
       gamma=9.52308795761162, importance_type='gain',
       learning_rate=0.16537402362986742, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=17, n_jobs=-1,
       nthread=None, nthreads=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9837701891769275, verbosity=1)
[16:06:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Time Taken =  61.2

In [16]:
# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))

[16:06:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 
 
Model performance on testing data
Mean Absolute Error: 66.09869929092885
Root Mean Squared Error: 118.75969887205845
r2: 0.5566621246892616


## Neural Network

##### Shallow Network

In [48]:
def build_model():
    model = Sequential()
    model.add(Dense(units=44, activation = 'relu', input_dim=22))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 86408.9516 - mae: 205.0695 - mse: 86408.9375
Epoch 2/100
600/600 [==============================] - 0s 110us/step - loss: 86231.3344 - mae: 204.6306 - mse: 86231.3438
Epoch 3/100
600/600 [==============================] - 0s 105us/step - loss: 86057.6057 - mae: 204.1972 - mse: 86057.6172
Epoch 4/100
600/600 [==============================] - 0s 118us/step - loss: 85872.2539 - mae: 203.7346 - mse: 85872.2500
Epoch 5/100
600/600 [==============================] - 0s 121us/step - loss: 85663.6621 - mae: 203.2155 - mse: 85663.6562
Epoch 6/100
600/600 [==============================] - 0s 105us/step - loss: 85418.0018 - mae: 202.6086 - mse: 85417.9922
Epoch 7/100
600/600 [==============================] - 0s 121us/step - loss: 85138.1362 - mae: 201.9070 - mse: 85138.1328
Epoch 8/100
600/600 [==============================] - 0s 110us/step - loss: 84814.7635 - mae: 201.1113 - mse: 84814.7500
Epoch 9/100
600/600 [=====

600/600 [==============================] - 0s 111us/step - loss: 47367.1505 - mae: 122.9415 - mse: 47367.1484
Epoch 68/100
600/600 [==============================] - 0s 113us/step - loss: 47187.7520 - mae: 123.1646 - mse: 47187.7500
Epoch 69/100
600/600 [==============================] - 0s 115us/step - loss: 47032.3105 - mae: 123.4492 - mse: 47032.3086
Epoch 70/100
600/600 [==============================] - 0s 110us/step - loss: 46883.9021 - mae: 123.7406 - mse: 46883.8984
Epoch 71/100
600/600 [==============================] - 0s 95us/step - loss: 46757.9192 - mae: 124.0537 - mse: 46757.9180
Epoch 72/100
600/600 [==============================] - 0s 107us/step - loss: 46635.8461 - mae: 124.3782 - mse: 46635.8477
Epoch 73/100
600/600 [==============================] - 0s 110us/step - loss: 46518.8475 - mae: 124.5692 - mse: 46518.8516
Epoch 74/100
600/600 [==============================] - 0s 110us/step - loss: 46410.2217 - mae: 124.8043 - mse: 46410.2227
Epoch 75/100
600/600 [========

600/600 [==============================] - 0s 116us/step - loss: 64024.8953 - mae: 143.9886 - mse: 64024.8945
Epoch 34/100
600/600 [==============================] - 0s 119us/step - loss: 63013.1906 - mae: 141.7873 - mse: 63013.1875
Epoch 35/100
600/600 [==============================] - 0s 134us/step - loss: 62021.7443 - mae: 139.6353 - mse: 62021.7461
Epoch 36/100
600/600 [==============================] - 0s 116us/step - loss: 61074.7951 - mae: 137.5847 - mse: 61074.8008
Epoch 37/100
600/600 [==============================] - 0s 113us/step - loss: 60171.2630 - mae: 135.6227 - mse: 60171.2617
Epoch 38/100
600/600 [==============================] - 0s 111us/step - loss: 59269.5309 - mae: 133.7991 - mse: 59269.5391
Epoch 39/100
600/600 [==============================] - 0s 110us/step - loss: 58470.4533 - mae: 132.0267 - mse: 58470.4609
Epoch 40/100
600/600 [==============================] - 0s 110us/step - loss: 57664.8647 - mae: 130.4414 - mse: 57664.8672
Epoch 41/100
600/600 [=======

600/600 [==============================] - 0s 123us/step - loss: 46630.1695 - mae: 128.1787 - mse: 46630.1680
Epoch 100/100
600/600 [==============================] - 0s 115us/step - loss: 46573.3979 - mae: 128.1124 - mse: 46573.3984
Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 78031.5372 - mae: 195.4512 - mse: 78031.5312
Epoch 2/100
600/600 [==============================] - 0s 121us/step - loss: 77900.8827 - mae: 195.1091 - mse: 77900.8828
Epoch 3/100
600/600 [==============================] - 0s 113us/step - loss: 77756.4608 - mae: 194.7412 - mse: 77756.4609
Epoch 4/100
600/600 [==============================] - 0s 111us/step - loss: 77589.6331 - mae: 194.3027 - mse: 77589.6328
Epoch 5/100
600/600 [==============================] - 0s 123us/step - loss: 77380.4841 - mae: 193.7699 - mse: 77380.4844
Epoch 6/100
600/600 [==============================] - 0s 116us/step - loss: 77128.6013 - mae: 193.1083 - mse: 77128.6094
Epoch 7/100
600/600 [===============

600/600 [==============================] - 0s 116us/step - loss: 42513.3040 - mae: 113.4993 - mse: 42513.3047
Epoch 66/100
600/600 [==============================] - 0s 95us/step - loss: 42353.4087 - mae: 113.6662 - mse: 42353.4102
Epoch 67/100
600/600 [==============================] - 0s 117us/step - loss: 42222.2393 - mae: 113.9107 - mse: 42222.2383
Epoch 68/100
600/600 [==============================] - 0s 126us/step - loss: 42084.7783 - mae: 114.1310 - mse: 42084.7734
Epoch 69/100
600/600 [==============================] - 0s 115us/step - loss: 41968.3167 - mae: 114.4181 - mse: 41968.3125
Epoch 70/100
600/600 [==============================] - 0s 93us/step - loss: 41858.0033 - mae: 114.6479 - mse: 41858.0039
Epoch 71/100
600/600 [==============================] - 0s 115us/step - loss: 41750.1720 - mae: 114.8549 - mse: 41750.1680
Epoch 72/100
600/600 [==============================] - 0s 116us/step - loss: 41660.3246 - mae: 115.1229 - mse: 41660.3242
Epoch 73/100
600/600 [=========

600/600 [==============================] - 0s 120us/step - loss: 65468.4062 - mae: 146.2320 - mse: 65468.4062
Epoch 32/100
600/600 [==============================] - 0s 113us/step - loss: 64550.2102 - mae: 144.0920 - mse: 64550.2148
Epoch 33/100
600/600 [==============================] - 0s 113us/step - loss: 63628.9897 - mae: 142.0658 - mse: 63628.9922
Epoch 34/100
600/600 [==============================] - 0s 116us/step - loss: 62771.7020 - mae: 140.0671 - mse: 62771.6992
Epoch 35/100
600/600 [==============================] - 0s 103us/step - loss: 61888.5656 - mae: 138.0858 - mse: 61888.5586
Epoch 36/100
600/600 [==============================] - 0s 120us/step - loss: 61036.6160 - mae: 136.2916 - mse: 61036.6211
Epoch 37/100
600/600 [==============================] - 0s 120us/step - loss: 60214.8934 - mae: 134.4348 - mse: 60214.8984
Epoch 38/100
600/600 [==============================] - 0s 116us/step - loss: 59436.3815 - mae: 132.7569 - mse: 59436.3789
Epoch 39/100
600/600 [=======

600/600 [==============================] - 0s 118us/step - loss: 46353.3715 - mae: 126.2851 - mse: 46353.3672
Epoch 98/100
600/600 [==============================] - 0s 130us/step - loss: 46298.0714 - mae: 126.2777 - mse: 46298.0742
Epoch 99/100
600/600 [==============================] - 0s 113us/step - loss: 46241.4316 - mae: 126.2400 - mse: 46241.4375
Epoch 100/100
600/600 [==============================] - 0s 121us/step - loss: 46188.5081 - mae: 126.2360 - mse: 46188.5117
Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 83523.1938 - mae: 203.0974 - mse: 83523.1953
Epoch 2/100
600/600 [==============================] - 0s 116us/step - loss: 83368.3693 - mae: 202.7131 - mse: 83368.3672
Epoch 3/100
600/600 [==============================] - 0s 120us/step - loss: 83186.4378 - mae: 202.2709 - mse: 83186.4453
Epoch 4/100
600/600 [==============================] - 0s 117us/step - loss: 82962.8467 - mae: 201.7295 - mse: 82962.8516
Epoch 5/100
600/600 [=============

600/600 [==============================] - 0s 131us/step - loss: 44139.1642 - mae: 121.9798 - mse: 44139.1641
Epoch 64/100
600/600 [==============================] - 0s 98us/step - loss: 44015.6619 - mae: 122.1960 - mse: 44015.6641
Epoch 65/100
600/600 [==============================] - 0s 101us/step - loss: 43924.9560 - mae: 122.4251 - mse: 43924.9609
Epoch 66/100
600/600 [==============================] - 0s 121us/step - loss: 43815.0268 - mae: 122.6211 - mse: 43815.0312
Epoch 67/100
600/600 [==============================] - 0s 100us/step - loss: 43743.8632 - mae: 122.9047 - mse: 43743.8633
Epoch 68/100
600/600 [==============================] - 0s 115us/step - loss: 43646.5900 - mae: 123.2193 - mse: 43646.5859
Epoch 69/100
600/600 [==============================] - 0s 97us/step - loss: 43564.2456 - mae: 123.3585 - mse: 43564.2461
Epoch 70/100
600/600 [==============================] - 0s 106us/step - loss: 43491.7260 - mae: 123.5540 - mse: 43491.7266
Epoch 71/100
600/600 [=========

750/750 [==============================] - 0s 104us/step - loss: 58074.4727 - mae: 135.4175 - mse: 58074.4805
Epoch 30/100
750/750 [==============================] - 0s 84us/step - loss: 57054.2667 - mae: 133.1459 - mse: 57054.2734
Epoch 31/100
750/750 [==============================] - 0s 97us/step - loss: 56054.6070 - mae: 131.0011 - mse: 56054.6133
Epoch 32/100
750/750 [==============================] - 0s 122us/step - loss: 55092.1847 - mae: 128.9374 - mse: 55092.1797
Epoch 33/100
750/750 [==============================] - 0s 90us/step - loss: 54180.1234 - mae: 127.1163 - mse: 54180.1172
Epoch 34/100
750/750 [==============================] - 0s 123us/step - loss: 53346.9783 - mae: 125.3175 - mse: 53346.9805
Epoch 35/100
750/750 [==============================] - 0s 106us/step - loss: 52550.4298 - mae: 124.0093 - mse: 52550.4375
Epoch 36/100
750/750 [==============================] - 0s 123us/step - loss: 51794.9271 - mae: 122.8790 - mse: 51794.9336
Epoch 37/100
750/750 [==========

750/750 [==============================] - 0s 122us/step - loss: 42735.2954 - mae: 123.2876 - mse: 42735.2969
Epoch 96/100
750/750 [==============================] - 0s 112us/step - loss: 42657.1660 - mae: 123.2964 - mse: 42657.1641
Epoch 97/100
750/750 [==============================] - 0s 116us/step - loss: 42589.8319 - mae: 123.1308 - mse: 42589.8281
Epoch 98/100
750/750 [==============================] - 0s 109us/step - loss: 42514.4216 - mae: 123.0910 - mse: 42514.4141
Epoch 99/100
750/750 [==============================] - 0s 128us/step - loss: 42438.6078 - mae: 123.0540 - mse: 42438.6094
Epoch 100/100
750/750 [==============================] - 0s 114us/step - loss: 42365.5973 - mae: 122.8712 - mse: 42365.6016


In [50]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[-41914.73196494 -32352.36187907 -61978.81451279 -34767.00028121
 -51511.55042781]
 
Mean and variance: 44504.89 (+/- 21975.45)
 
 
Model performance on testing data
Mean Absolute Error: 140.13280337402344
Root Mean Squared Error: 247.63615895745144
Time Taken =  137.84375


##### Medium Deep Network

In [44]:
def build_model():
    model = Sequential()
    model.add(Dense(units=22, activation = 'relu', input_dim=22))
    model.add(Dense(units=22, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 86025.2585 - mae: 204.1486 - mse: 86025.2578
Epoch 2/100
600/600 [==============================] - 0s 103us/step - loss: 85810.5232 - mae: 203.6063 - mse: 85810.5234
Epoch 3/100
600/600 [==============================] - 0s 100us/step - loss: 85524.5107 - mae: 202.9254 - mse: 85524.5156
Epoch 4/100
600/600 [==============================] - 0s 95us/step - loss: 85104.1729 - mae: 201.9057 - mse: 85104.1797
Epoch 5/100
600/600 [==============================] - 0s 91us/step - loss: 84472.1242 - mae: 200.3135 - mse: 84472.1172
Epoch 6/100
600/600 [==============================] - 0s 70us/step - loss: 83519.0888 - mae: 197.9651 - mse: 83519.0938
Epoch 7/100
600/600 [==============================] - 0s 90us/step - loss: 82207.0527 - mae: 194.5683 - mse: 82207.0547
Epoch 8/100
600/600 [==============================] - 0s 110us/step - loss: 80349.8766 - mae: 190.0406 - mse: 80349.8828
Epoch 9/100
600/600 [=========

600/600 [==============================] - 0s 106us/step - loss: 36976.8757 - mae: 117.1197 - mse: 36976.8750
Epoch 69/100
600/600 [==============================] - 0s 111us/step - loss: 36726.5620 - mae: 116.8241 - mse: 36726.5625
Epoch 70/100
600/600 [==============================] - 0s 94us/step - loss: 36497.9427 - mae: 116.4123 - mse: 36497.9414
Epoch 71/100
600/600 [==============================] - 0s 95us/step - loss: 36236.8722 - mae: 115.8274 - mse: 36236.8750
Epoch 72/100
600/600 [==============================] - 0s 105us/step - loss: 36017.0533 - mae: 115.5790 - mse: 36017.0547
Epoch 73/100
600/600 [==============================] - 0s 108us/step - loss: 35757.7006 - mae: 114.8851 - mse: 35757.6992
Epoch 74/100
600/600 [==============================] - 0s 105us/step - loss: 35520.7813 - mae: 114.1862 - mse: 35520.7852
Epoch 75/100
600/600 [==============================] - 0s 116us/step - loss: 35306.3555 - mae: 114.2244 - mse: 35306.3555
Epoch 76/100
600/600 [=========

Epoch 35/100
600/600 [==============================] - 0s 110us/step - loss: 46730.5284 - mae: 129.5389 - mse: 46730.5312
Epoch 36/100
600/600 [==============================] - 0s 105us/step - loss: 46584.6176 - mae: 129.5682 - mse: 46584.6172
Epoch 37/100
600/600 [==============================] - 0s 100us/step - loss: 46419.0508 - mae: 128.8889 - mse: 46419.0547
Epoch 38/100
600/600 [==============================] - 0s 111us/step - loss: 46249.7536 - mae: 128.5937 - mse: 46249.7617
Epoch 39/100
600/600 [==============================] - 0s 108us/step - loss: 46098.2464 - mae: 128.6640 - mse: 46098.2461
Epoch 40/100
600/600 [==============================] - 0s 105us/step - loss: 45925.4208 - mae: 128.7363 - mse: 45925.4141
Epoch 41/100
600/600 [==============================] - 0s 108us/step - loss: 45782.5098 - mae: 128.7450 - mse: 45782.5117
Epoch 42/100
600/600 [==============================] - 0s 93us/step - loss: 45606.1266 - mae: 128.1819 - mse: 45606.1250
Epoch 43/100
600/

600/600 [==============================] - 1s 1ms/step - loss: 77803.9430 - mae: 194.8648 - mse: 77803.9453
Epoch 2/100
600/600 [==============================] - 0s 110us/step - loss: 77634.9992 - mae: 194.4208 - mse: 77635.0078
Epoch 3/100
600/600 [==============================] - 0s 106us/step - loss: 77388.3615 - mae: 193.7826 - mse: 77388.3594
Epoch 4/100
600/600 [==============================] - 0s 93us/step - loss: 77030.3457 - mae: 192.8145 - mse: 77030.3516
Epoch 5/100
600/600 [==============================] - 0s 96us/step - loss: 76493.0771 - mae: 191.3977 - mse: 76493.0703
Epoch 6/100
600/600 [==============================] - 0s 91us/step - loss: 75711.8935 - mae: 189.3199 - mse: 75711.8906
Epoch 7/100
600/600 [==============================] - 0s 93us/step - loss: 74592.9849 - mae: 186.3808 - mse: 74592.9922
Epoch 8/100
600/600 [==============================] - 0s 92us/step - loss: 73111.9357 - mae: 182.4806 - mse: 73111.9297
Epoch 9/100
600/600 [======================

Epoch 68/100
600/600 [==============================] - 0s 105us/step - loss: 34575.4179 - mae: 111.5041 - mse: 34575.4180
Epoch 69/100
600/600 [==============================] - 0s 111us/step - loss: 34387.8380 - mae: 111.2783 - mse: 34387.8398
Epoch 70/100
600/600 [==============================] - 0s 95us/step - loss: 34167.0758 - mae: 110.7965 - mse: 34167.0781
Epoch 71/100
600/600 [==============================] - 0s 105us/step - loss: 33973.9263 - mae: 110.3912 - mse: 33973.9258
Epoch 72/100
600/600 [==============================] - 0s 105us/step - loss: 33760.5988 - mae: 109.8293 - mse: 33760.6016
Epoch 73/100
600/600 [==============================] - 0s 93us/step - loss: 33595.7555 - mae: 108.7980 - mse: 33595.7578
Epoch 74/100
600/600 [==============================] - 0s 95us/step - loss: 33379.6081 - mae: 108.7910 - mse: 33379.6055
Epoch 75/100
600/600 [==============================] - 0s 95us/step - loss: 33153.2620 - mae: 108.9047 - mse: 33153.2617
Epoch 76/100
600/600

Epoch 35/100
600/600 [==============================] - 0s 85us/step - loss: 45804.5935 - mae: 126.8667 - mse: 45804.5938
Epoch 36/100
600/600 [==============================] - 0s 112us/step - loss: 45632.9591 - mae: 126.8902 - mse: 45632.9609
Epoch 37/100
600/600 [==============================] - 0s 123us/step - loss: 45447.6569 - mae: 126.9571 - mse: 45447.6602
Epoch 38/100
600/600 [==============================] - 0s 87us/step - loss: 45289.6397 - mae: 127.1084 - mse: 45289.6406
Epoch 39/100
600/600 [==============================] - 0s 74us/step - loss: 45094.9302 - mae: 126.8247 - mse: 45094.9219
Epoch 40/100
600/600 [==============================] - 0s 93us/step - loss: 44912.1128 - mae: 125.9709 - mse: 44912.1133
Epoch 41/100
600/600 [==============================] - 0s 87us/step - loss: 44732.9402 - mae: 126.0192 - mse: 44732.9414
Epoch 42/100
600/600 [==============================] - 0s 85us/step - loss: 44534.2868 - mae: 126.0712 - mse: 44534.2852
Epoch 43/100
600/600 [

Epoch 2/100
600/600 [==============================] - 0s 78us/step - loss: 83482.1840 - mae: 202.9945 - mse: 83482.1875
Epoch 3/100
600/600 [==============================] - 0s 86us/step - loss: 83357.6857 - mae: 202.6846 - mse: 83357.6875
Epoch 4/100
600/600 [==============================] - 0s 94us/step - loss: 83181.7129 - mae: 202.2555 - mse: 83181.7188
Epoch 5/100
600/600 [==============================] - 0s 105us/step - loss: 82929.9260 - mae: 201.6180 - mse: 82929.9297
Epoch 6/100
600/600 [==============================] - 0s 111us/step - loss: 82540.2607 - mae: 200.6531 - mse: 82540.2578
Epoch 7/100
600/600 [==============================] - 0s 103us/step - loss: 81951.6526 - mae: 199.2050 - mse: 81951.6562
Epoch 8/100
600/600 [==============================] - 0s 106us/step - loss: 81118.7674 - mae: 197.0938 - mse: 81118.7734
Epoch 9/100
600/600 [==============================] - 0s 109us/step - loss: 79934.9260 - mae: 194.0761 - mse: 79934.9297
Epoch 10/100
600/600 [=====

600/600 [==============================] - 0s 110us/step - loss: 35566.7340 - mae: 117.1784 - mse: 35566.7305
Epoch 69/100
600/600 [==============================] - 0s 106us/step - loss: 35365.7837 - mae: 116.4903 - mse: 35365.7852
Epoch 70/100
600/600 [==============================] - 0s 101us/step - loss: 35140.1255 - mae: 116.1018 - mse: 35140.1250
Epoch 71/100
600/600 [==============================] - 0s 113us/step - loss: 34984.8938 - mae: 116.5307 - mse: 34984.8984
Epoch 72/100
600/600 [==============================] - 0s 121us/step - loss: 34690.0617 - mae: 115.5727 - mse: 34690.0586
Epoch 73/100
600/600 [==============================] - 0s 113us/step - loss: 34481.3514 - mae: 114.7421 - mse: 34481.3516
Epoch 74/100
600/600 [==============================] - 0s 115us/step - loss: 34252.3579 - mae: 114.3160 - mse: 34252.3555
Epoch 75/100
600/600 [==============================] - 0s 113us/step - loss: 34022.9966 - mae: 114.3201 - mse: 34022.9961
Epoch 76/100
600/600 [=======

Epoch 35/100
750/750 [==============================] - 0s 106us/step - loss: 42220.2490 - mae: 123.4877 - mse: 42220.2500
Epoch 36/100
750/750 [==============================] - 0s 108us/step - loss: 42012.0096 - mae: 123.5375 - mse: 42012.0117
Epoch 37/100
750/750 [==============================] - 0s 105us/step - loss: 41808.6521 - mae: 123.7586 - mse: 41808.6523
Epoch 38/100
750/750 [==============================] - 0s 107us/step - loss: 41609.1574 - mae: 123.4319 - mse: 41609.1602
Epoch 39/100
750/750 [==============================] - 0s 101us/step - loss: 41394.3899 - mae: 123.1841 - mse: 41394.3867
Epoch 40/100
750/750 [==============================] - 0s 106us/step - loss: 41186.2883 - mae: 122.5807 - mse: 41186.2930
Epoch 41/100
750/750 [==============================] - 0s 88us/step - loss: 40970.1842 - mae: 122.2040 - mse: 40970.1836
Epoch 42/100
750/750 [==============================] - 0s 100us/step - loss: 40759.6897 - mae: 121.9365 - mse: 40759.6914
Epoch 43/100
750/

In [45]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[-28341.99958333 -20514.16307222 -45200.4692771  -24950.09222781
 -39039.25112159]
 
Mean and variance: 31609.20 (+/- 18287.62)
 
 
Model performance on testing data
Mean Absolute Error: 108.65183544799805
Root Mean Squared Error: 204.58919486071133
Time Taken =  120.703125


##### Deep Network

In [46]:
def build_model():
    model = Sequential()
    model.add(Dense(units=11, activation = 'relu', input_dim=22))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 86356.1073 - mae: 204.9122 - mse: 86356.1094
Epoch 2/100
600/600 [==============================] - 0s 115us/step - loss: 86323.5849 - mae: 204.8364 - mse: 86323.5859
Epoch 3/100
600/600 [==============================] - 0s 106us/step - loss: 86297.2849 - mae: 204.7706 - mse: 86297.2812
Epoch 4/100
600/600 [==============================] - 0s 125us/step - loss: 86260.7523 - mae: 204.6793 - mse: 86260.7578
Epoch 5/100
600/600 [==============================] - 0s 123us/step - loss: 86195.3736 - mae: 204.5128 - mse: 86195.3828
Epoch 6/100
600/600 [==============================] - 0s 112us/step - loss: 86083.9477 - mae: 204.2383 - mse: 86083.9531
Epoch 7/100
600/600 [==============================] - 0s 110us/step - loss: 85910.7497 - mae: 203.8105 - mse: 85910.7500
Epoch 8/100
600/600 [==============================] - 0s 108us/step - loss: 85641.3262 - mae: 203.1542 - mse: 85641.3359
Epoch 9/100
600/600 [=====

600/600 [==============================] - 0s 139us/step - loss: 32889.2843 - mae: 109.9963 - mse: 32889.2852
Epoch 68/100
600/600 [==============================] - 0s 112us/step - loss: 32531.3025 - mae: 110.2185 - mse: 32531.3027
Epoch 69/100
600/600 [==============================] - 0s 105us/step - loss: 32135.9141 - mae: 109.8215 - mse: 32135.9160
Epoch 70/100
600/600 [==============================] - 0s 89us/step - loss: 31780.3652 - mae: 109.0644 - mse: 31780.3672
Epoch 71/100
600/600 [==============================] - 0s 94us/step - loss: 31402.6375 - mae: 107.6368 - mse: 31402.6406
Epoch 72/100
600/600 [==============================] - 0s 90us/step - loss: 31011.9435 - mae: 107.0061 - mse: 31011.9395
Epoch 73/100
600/600 [==============================] - 0s 108us/step - loss: 30648.0948 - mae: 106.2948 - mse: 30648.0957
Epoch 74/100
600/600 [==============================] - 0s 91us/step - loss: 30333.5232 - mae: 106.6049 - mse: 30333.5234
Epoch 75/100
600/600 [===========

600/600 [==============================] - 0s 93us/step - loss: 46393.9460 - mae: 129.2937 - mse: 46393.9414
Epoch 35/100
600/600 [==============================] - 0s 86us/step - loss: 46236.9781 - mae: 129.0549 - mse: 46236.9805
Epoch 36/100
600/600 [==============================] - 0s 116us/step - loss: 46069.5365 - mae: 128.9280 - mse: 46069.5391
Epoch 37/100
600/600 [==============================] - 0s 118us/step - loss: 45918.7681 - mae: 128.8362 - mse: 45918.7695
Epoch 38/100
600/600 [==============================] - 0s 106us/step - loss: 45762.4768 - mae: 128.6837 - mse: 45762.4727
Epoch 39/100
600/600 [==============================] - 0s 116us/step - loss: 45579.4806 - mae: 128.4513 - mse: 45579.4805
Epoch 40/100
600/600 [==============================] - 0s 113us/step - loss: 45441.7250 - mae: 128.0968 - mse: 45441.7266
Epoch 41/100
600/600 [==============================] - 0s 83us/step - loss: 45255.9232 - mae: 128.0750 - mse: 45255.9219
Epoch 42/100
600/600 [==========

600/600 [==============================] - 0s 118us/step - loss: 30001.5000 - mae: 105.5575 - mse: 30001.5000
Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 77962.6747 - mae: 195.2872 - mse: 77962.6797
Epoch 2/100
600/600 [==============================] - 0s 89us/step - loss: 77876.5219 - mae: 195.0615 - mse: 77876.5234
Epoch 3/100
600/600 [==============================] - 0s 99us/step - loss: 77770.6243 - mae: 194.7799 - mse: 77770.6328
Epoch 4/100
600/600 [==============================] - 0s 115us/step - loss: 77631.2676 - mae: 194.4128 - mse: 77631.2656
Epoch 5/100
600/600 [==============================] - 0s 113us/step - loss: 77445.1771 - mae: 193.9141 - mse: 77445.1875
Epoch 6/100
600/600 [==============================] - 0s 111us/step - loss: 77186.5087 - mae: 193.2658 - mse: 77186.5078
Epoch 7/100
600/600 [==============================] - 0s 105us/step - loss: 76863.8456 - mae: 192.3954 - mse: 76863.8438
Epoch 8/100
600/600 [===================

600/600 [==============================] - 0s 121us/step - loss: 35938.4968 - mae: 113.0703 - mse: 35938.5000
Epoch 67/100
600/600 [==============================] - 0s 142us/step - loss: 35716.7137 - mae: 112.5146 - mse: 35716.7109
Epoch 68/100
600/600 [==============================] - 0s 112us/step - loss: 35503.5122 - mae: 112.6555 - mse: 35503.5117
Epoch 69/100
600/600 [==============================] - 0s 110us/step - loss: 35254.3422 - mae: 112.1756 - mse: 35254.3398
Epoch 70/100
600/600 [==============================] - 0s 124us/step - loss: 35034.7629 - mae: 111.7096 - mse: 35034.7656
Epoch 71/100
600/600 [==============================] - 0s 123us/step - loss: 34808.3533 - mae: 111.3667 - mse: 34808.3516
Epoch 72/100
600/600 [==============================] - 0s 110us/step - loss: 34552.6516 - mae: 110.8504 - mse: 34552.6484
Epoch 73/100
600/600 [==============================] - 0s 90us/step - loss: 34318.4408 - mae: 110.9118 - mse: 34318.4453
Epoch 74/100
600/600 [========

600/600 [==============================] - 0s 109us/step - loss: 45294.0059 - mae: 126.1099 - mse: 45294.0117
Epoch 34/100
600/600 [==============================] - 0s 102us/step - loss: 45062.8224 - mae: 126.0192 - mse: 45062.8242
Epoch 35/100
600/600 [==============================] - 0s 92us/step - loss: 44884.7356 - mae: 126.8592 - mse: 44884.7383
Epoch 36/100
600/600 [==============================] - 0s 87us/step - loss: 44571.4048 - mae: 126.3523 - mse: 44571.4023
Epoch 37/100
600/600 [==============================] - 0s 114us/step - loss: 44291.3494 - mae: 125.2505 - mse: 44291.3477
Epoch 38/100
600/600 [==============================] - 0s 93us/step - loss: 44016.5123 - mae: 125.0322 - mse: 44016.5117
Epoch 39/100
600/600 [==============================] - 0s 121us/step - loss: 43770.6522 - mae: 125.2141 - mse: 43770.6484
Epoch 40/100
600/600 [==============================] - 0s 111us/step - loss: 43494.0576 - mae: 124.7622 - mse: 43494.0586
Epoch 41/100
600/600 [==========

600/600 [==============================] - 0s 75us/step - loss: 23635.1897 - mae: 94.9152 - mse: 23635.1914
Epoch 1/100
600/600 [==============================] - 1s 1ms/step - loss: 83525.4167 - mae: 203.1087 - mse: 83525.4141
Epoch 2/100
600/600 [==============================] - 0s 91us/step - loss: 83448.7408 - mae: 202.9219 - mse: 83448.7422
Epoch 3/100
600/600 [==============================] - 0s 89us/step - loss: 83332.2885 - mae: 202.6349 - mse: 83332.2891
Epoch 4/100
600/600 [==============================] - 0s 118us/step - loss: 83166.6508 - mae: 202.2181 - mse: 83166.6562
Epoch 5/100
600/600 [==============================] - 0s 106us/step - loss: 82923.3391 - mae: 201.6104 - mse: 82923.3281
Epoch 6/100
600/600 [==============================] - 0s 105us/step - loss: 82552.7328 - mae: 200.6882 - mse: 82552.7266
Epoch 7/100
600/600 [==============================] - 0s 138us/step - loss: 81957.3596 - mae: 199.2587 - mse: 81957.3516
Epoch 8/100
600/600 [=====================

600/600 [==============================] - 0s 80us/step - loss: 30989.2750 - mae: 110.2937 - mse: 30989.2734
Epoch 68/100
600/600 [==============================] - 0s 108us/step - loss: 30591.4564 - mae: 109.7550 - mse: 30591.4570
Epoch 69/100
600/600 [==============================] - 0s 115us/step - loss: 30205.1090 - mae: 109.2184 - mse: 30205.1094
Epoch 70/100
600/600 [==============================] - 0s 121us/step - loss: 29884.3911 - mae: 107.0861 - mse: 29884.3906
Epoch 71/100
600/600 [==============================] - 0s 103us/step - loss: 29470.2366 - mae: 106.3554 - mse: 29470.2363
Epoch 72/100
600/600 [==============================] - 0s 112us/step - loss: 29101.9363 - mae: 106.9313 - mse: 29101.9336
Epoch 73/100
600/600 [==============================] - 0s 73us/step - loss: 28725.5866 - mae: 106.2684 - mse: 28725.5859
Epoch 74/100
600/600 [==============================] - 0s 90us/step - loss: 28370.2200 - mae: 105.4698 - mse: 28370.2207
Epoch 75/100
600/600 [==========

750/750 [==============================] - 0s 130us/step - loss: 42064.9995 - mae: 124.0557 - mse: 42065.0000
Epoch 35/100
750/750 [==============================] - 0s 128us/step - loss: 41823.5826 - mae: 124.2617 - mse: 41823.5820
Epoch 36/100
750/750 [==============================] - 0s 104us/step - loss: 41549.0570 - mae: 123.3421 - mse: 41549.0508
Epoch 37/100
750/750 [==============================] - 0s 101us/step - loss: 41278.7684 - mae: 123.7529 - mse: 41278.7695
Epoch 38/100
750/750 [==============================] - 0s 93us/step - loss: 40951.2201 - mae: 122.5716 - mse: 40951.2227
Epoch 39/100
750/750 [==============================] - 0s 100us/step - loss: 40646.5522 - mae: 121.6004 - mse: 40646.5508
Epoch 40/100
750/750 [==============================] - 0s 117us/step - loss: 40322.0241 - mae: 121.5864 - mse: 40322.0273
Epoch 41/100
750/750 [==============================] - 0s 108us/step - loss: 39995.2533 - mae: 121.3368 - mse: 39995.2539
Epoch 42/100
750/750 [========

In [47]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[-22029.25380159 -18509.39726945 -42781.02495654 -25714.56728207
 -32828.6549837 ]
 
Mean and variance: 28372.58 (+/- 17252.13)
 
 
Model performance on testing data
Mean Absolute Error: 110.56946370689393
Root Mean Squared Error: 196.82085029686246
Time Taken =  134.640625


## Part B

In [23]:
df_purchase = df[df['Purchase'] == 1]

# Selecting required columns
X_df = df_purchase.iloc[:,1:23]
y_df = df_purchase.iloc[:,24:]

##### Creating the Train and Test split

In [24]:
X_train, X_test_holdout, y_train, y_test_holdout = train_test_split(X_df, y_df, test_size=0.25, random_state=42)

##### Normalizing data

In [25]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

x_test_scaled = scaler.fit_transform(X_test_holdout)
X_test_holdout = pd.DataFrame(x_test_scaled)

### Linear Regression

In [26]:
start = time.process_time()
# create linear regression object 
lr = linear_model.LinearRegression()

# cross validation 
scores = cross_val_score(lr, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

print("Below are the mean squared errors for each model run")
print(scores)
print(" ")
print("Mean score: %0.2f " % (abs(scores.mean())))

# Model fit on training data and predicting on testing data
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test_holdout)

# Model performance on testing data
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('Explained Variance:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print(" ")
print('Time Taken = ', time.process_time() - start)

Below are the mean squared errors for each model run
[-19812.07066791 -12602.97017752 -34733.42026459 -25786.96103861
 -28542.89446855]
 
Mean score: 24295.66 
 
 
Model performance on testing data
Mean Absolute Error: 110.57948435806942
Root Mean Squared Error: 191.2238772941838
r2: 0.41548093167861555
 
Time Taken =  0.140625


### Lasso Regression

In [27]:
start = time.process_time()

lasso = linear_model.Lasso()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_lasso = GridSearchCV(lasso, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_lasso.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_lasso.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_lasso.best_params_)

print(" ")
print("Best estimator")
print (grid_lasso.best_estimator_)

# predicting on test data
lasso = grid_lasso.best_estimator_
lasso.fit(X_train,y_train)
y_pred = lasso.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  23951.402393936794
 
Best parameters
{'alpha': 0.5000000000000001}
 
Best estimator
Lasso(alpha=0.5000000000000001, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 109.7550939894061
Root Mean Squared Error: 190.59275614275074
r2: 0.419332893937013
Time Taken =  0.9375


### Ridge Regression

In [28]:
start = time.process_time()

ridge = linear_model.Ridge()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_ridge = GridSearchCV(ridge, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_ridge.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_ridge.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_ridge.best_params_)

print(" ")
print("Best estimator")
print (grid_ridge.best_estimator_)

# predicting on test data
ridge = grid_ridge.best_estimator_
ridge.fit(X_train,y_train)
y_pred = ridge.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  24249.094866401145
 
Best parameters
{'alpha': 0.30000000000000004}
 
Best estimator
Ridge(alpha=0.30000000000000004, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)
 
 
Model performance on testing data
Mean Absolute Error: 110.13709948664578
Root Mean Squared Error: 191.4901826660552
r2: 0.4138517527354626
Time Taken =  1.390625


### KNN

In [29]:
start = time.process_time()

knn = neighbors.KNeighborsRegressor()

#Hyper Parameter tuning
param_set ={'n_neighbors': list(range(1,30)), 'weights': ["uniform", "distance"]}
grid_knn = GridSearchCV(knn, param_grid = param_set, cv=5, scoring='neg_mean_squared_error', verbose = 0)
grid_knn.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_knn.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_knn.best_params_)

print(" ")
print("Best estimator")
print (grid_knn.best_estimator_)

# predicting on test data
knn = grid_knn.best_estimator_
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  37913.72046451443
 
Best parameters
{'n_neighbors': 25, 'weights': 'distance'}
 
Best estimator
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=25, p=2,
          weights='distance')
 
 
Model performance on testing data
Mean Absolute Error: 123.70465389883117
Root Mean Squared Error: 240.02773522513584
r2: 0.07904735705996002
Time Taken =  6.9375


### Decision Tree

In [30]:
start = time.process_time()

DTree = tree.DecisionTreeRegressor()

#Hyper parameter tuning
param_set ={'max_depth': range(1,20), 'min_samples_split' : range(2,30)}
grid_DTree = GridSearchCV(DTree, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_DTree.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_DTree.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_DTree.best_params_)

print(" ")
print("Best estimator")
print (grid_DTree.best_estimator_)

# predicting on test data
DTree = grid_DTree.best_estimator_
DTree.fit(X_train,y_train)
y_pred = DTree.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  22323.240263277614
 
Best parameters
{'max_depth': 3, 'min_samples_split': 8}
 
Best estimator
DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
 
 
Model performance on testing data
Mean Absolute Error: 111.67511949161054
Root Mean Squared Error: 204.45116269214938
r2: 0.33181978305652526
Time Taken =  15.515625


### Support Vector Regression

In [31]:
start = time.process_time()

SVR = svm.SVR()
# Hyper parameter tuning using GridSearch
param_set = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1,2,10,100,1000], 'epsilon':[0.05,0.1,0.2,0.3,0.5]},
                    {'kernel': ['linear'], 'C': [1,2,5,10,100], 'epsilon':[0.05,0.1,0.2,0.3,0.5]}]
grid_SVR = GridSearchCV(SVR, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_SVR.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_SVR.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_SVR.best_params_)

print(" ")
print("Best estimator")
print (grid_SVR.best_estimator_)

# predicting on test data
SVR = grid_SVR.best_estimator_
SVR.fit(X_train,y_train)
y_pred = SVR.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  27815.104847457817
 
Best parameters
{'C': 100, 'epsilon': 0.05, 'kernel': 'linear'}
 
Best estimator
SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.05,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
 
 
Model performance on testing data
Mean Absolute Error: 108.22570555700518
Root Mean Squared Error: 210.49456562157204
r2: 0.32204375565882715
Time Taken =  12.484375


### Ensemble : Random Forest

In [32]:
start = time.process_time()
RF = ensemble.RandomForestRegressor()

# Hyper parameter tuning using GridSearch
param_set ={'max_depth': [3,10,20],
            'min_samples_split' :[4,5,10],
            'n_estimators': [100,250,500],
            'bootstrap':[True, False] ,
            'max_features':['auto','sqrt'],
            'n_jobs':[-1]
           }
grid_RF = GridSearchCV(RF, param_grid = param_set, cv=5, scoring='neg_mean_squared_error')
grid_RF.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_RF.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_RF.best_params_)

print(" ")
print("Best estimator")
print (grid_RF.best_estimator_)

# predicting on test data
RF = grid_RF.best_estimator_
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print(" ")
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  22323.240263277577
 
Best parameters
{'bootstrap': False, 'max_depth': 3, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 500, 'n_jobs': -1}
 
Best estimator
RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 111.6751194916104
Root Mean Squared Error: 204.4511626921494
r2: 0.33913672503684633
 
Time Taken =  168.734375


### Ensemble : XGBoost

In [34]:
start = time.process_time()
XGB = xgboost.XGBRegressor(nthreads=-1)

a = stat.beta(10, 1)

# Hyper parameter tuning using GridSearch
param_set ={"n_estimators": stat.randint(3, 40),
            "max_depth": stat.randint(3, 40),
            "learning_rate": stat.uniform(0.05, 0.4),
            "colsample_bytree": a,
            "subsample": a,
            "gamma": stat.uniform(0, 10),
            'n_jobs':[-1]
           }

grid_XGB = RandomizedSearchCV(XGB, param_set, cv=5, scoring='neg_mean_squared_error')
grid_XGB.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_XGB.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_XGB.best_params_)

print(" ")
print("Best estimator")
print (grid_XGB.best_estimator_)

# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
# print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
# print(" ")
print('Time Taken = ', time.process_time() - start)

[16:15:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:02] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:15:02] WARNI

Time Taken =  28.765625


In [35]:
# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))

[16:15:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 
 
Model performance on testing data
Mean Absolute Error: 106.66388892211916
Root Mean Squared Error: 195.5159751248307
r2: 0.4343583052403158


### Neural Network

##### Shallow Network

In [36]:
def build_model():
    model = Sequential()
    model.add(Dense(units=44, activation = 'relu', input_dim=22))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 0s 352us/step - loss: 86312.7380 - mae: 204.8103 - mse: 86312.7422
Epoch 2/100
600/600 [==============================] - 0s 27us/step - loss: 86165.4724 - mae: 204.4349 - mse: 86165.4766
Epoch 3/100
600/600 [==============================] - 0s 47us/step - loss: 86009.6742 - mae: 204.0499 - mse: 86009.6641
Epoch 4/100
600/600 [==============================] - 0s 38us/step - loss: 85837.3874 - mae: 203.6078 - mse: 85837.3828
Epoch 5/100
600/600 [==============================] - 0s 37us/step - loss: 85626.3576 - mae: 203.0801 - mse: 85626.3672
Epoch 6/100
600/600 [==============================] - 0s 42us/step - loss: 85367.0156 - mae: 202.4263 - mse: 85367.0234
Epoch 7/100
600/600 [==============================] - 0s 30us/step - loss: 85047.8618 - mae: 201.6194 - mse: 85047.8672
Epoch 8/100
600/600 [==============================] - 0s 42us/step - loss: 84658.6012 - mae: 200.6379 - mse: 84658.6016
Epoch 9/100
600/600 [==========

600/600 [==============================] - 0s 52us/step - loss: 46298.2104 - mae: 125.5671 - mse: 46298.2148
Epoch 68/100
600/600 [==============================] - 0s 53us/step - loss: 46191.1419 - mae: 125.7873 - mse: 46191.1406
Epoch 69/100
600/600 [==============================] - 0s 52us/step - loss: 46112.9514 - mae: 125.9605 - mse: 46112.9531
Epoch 70/100
600/600 [==============================] - 0s 50us/step - loss: 46044.9527 - mae: 126.1826 - mse: 46044.9492
Epoch 71/100
600/600 [==============================] - 0s 42us/step - loss: 45962.5404 - mae: 126.4732 - mse: 45962.5391
Epoch 72/100
600/600 [==============================] - 0s 48us/step - loss: 45895.0038 - mae: 126.6742 - mse: 45895.0039
Epoch 73/100
600/600 [==============================] - 0s 47us/step - loss: 45824.3249 - mae: 126.7887 - mse: 45824.3242
Epoch 74/100
600/600 [==============================] - 0s 47us/step - loss: 45763.4503 - mae: 126.9817 - mse: 45763.4492
Epoch 75/100
600/600 [===============

600/600 [==============================] - 0s 55us/step - loss: 65544.7279 - mae: 147.3941 - mse: 65544.7344
Epoch 34/100
600/600 [==============================] - 0s 47us/step - loss: 64581.5570 - mae: 145.2493 - mse: 64581.5586
Epoch 35/100
600/600 [==============================] - 0s 52us/step - loss: 63630.1773 - mae: 143.0657 - mse: 63630.1719
Epoch 36/100
600/600 [==============================] - 0s 50us/step - loss: 62714.2728 - mae: 141.0291 - mse: 62714.2812
Epoch 37/100
600/600 [==============================] - 0s 40us/step - loss: 61830.6302 - mae: 139.0674 - mse: 61830.6250
Epoch 38/100
600/600 [==============================] - 0s 48us/step - loss: 60915.9315 - mae: 137.1989 - mse: 60915.9336
Epoch 39/100
600/600 [==============================] - 0s 40us/step - loss: 60114.0504 - mae: 135.4655 - mse: 60114.0547
Epoch 40/100
600/600 [==============================] - 0s 47us/step - loss: 59287.5517 - mae: 133.7348 - mse: 59287.5469
Epoch 41/100
600/600 [===============

600/600 [==============================] - 0s 57us/step - loss: 46815.4653 - mae: 128.1585 - mse: 46815.4648
Epoch 1/100
600/600 [==============================] - 0s 346us/step - loss: 77882.1023 - mae: 195.0798 - mse: 77882.1016
Epoch 2/100
600/600 [==============================] - 0s 33us/step - loss: 77741.0499 - mae: 194.7161 - mse: 77741.0547
Epoch 3/100
600/600 [==============================] - 0s 40us/step - loss: 77595.8069 - mae: 194.3299 - mse: 77595.8047
Epoch 4/100
600/600 [==============================] - 0s 35us/step - loss: 77424.4982 - mae: 193.8947 - mse: 77424.4922
Epoch 5/100
600/600 [==============================] - 0s 40us/step - loss: 77220.6443 - mae: 193.3582 - mse: 77220.6484
Epoch 6/100
600/600 [==============================] - 0s 38us/step - loss: 76967.0259 - mae: 192.6930 - mse: 76967.0234
Epoch 7/100
600/600 [==============================] - 0s 35us/step - loss: 76656.2034 - mae: 191.8996 - mse: 76656.2031
Epoch 8/100
600/600 [======================

600/600 [==============================] - 0s 48us/step - loss: 41434.8449 - mae: 115.6004 - mse: 41434.8477
Epoch 68/100
600/600 [==============================] - 0s 53us/step - loss: 41341.0868 - mae: 115.8100 - mse: 41341.0859
Epoch 69/100
600/600 [==============================] - 0s 57us/step - loss: 41266.3604 - mae: 116.0602 - mse: 41266.3594
Epoch 70/100
600/600 [==============================] - 0s 55us/step - loss: 41189.1833 - mae: 116.2850 - mse: 41189.1836
Epoch 71/100
600/600 [==============================] - 0s 48us/step - loss: 41111.7324 - mae: 116.4637 - mse: 41111.7305
Epoch 72/100
600/600 [==============================] - 0s 37us/step - loss: 41048.4618 - mae: 116.5839 - mse: 41048.4609
Epoch 73/100
600/600 [==============================] - 0s 45us/step - loss: 40989.6834 - mae: 116.7810 - mse: 40989.6875
Epoch 74/100
600/600 [==============================] - 0s 50us/step - loss: 40928.1400 - mae: 116.9583 - mse: 40928.1406
Epoch 75/100
600/600 [===============

600/600 [==============================] - 0s 38us/step - loss: 61677.2746 - mae: 137.6780 - mse: 61677.2734
Epoch 35/100
600/600 [==============================] - 0s 42us/step - loss: 60800.8695 - mae: 135.8054 - mse: 60800.8672
Epoch 36/100
600/600 [==============================] - 0s 37us/step - loss: 59977.2275 - mae: 133.9401 - mse: 59977.2266
Epoch 37/100
600/600 [==============================] - 0s 43us/step - loss: 59138.0783 - mae: 132.1365 - mse: 59138.0781
Epoch 38/100
600/600 [==============================] - 0s 50us/step - loss: 58343.1003 - mae: 130.4042 - mse: 58343.1016
Epoch 39/100
600/600 [==============================] - 0s 46us/step - loss: 57586.4919 - mae: 128.8010 - mse: 57586.4922
Epoch 40/100
600/600 [==============================] - 0s 43us/step - loss: 56873.4746 - mae: 127.3652 - mse: 56873.4805
Epoch 41/100
600/600 [==============================] - 0s 45us/step - loss: 56152.9504 - mae: 125.9077 - mse: 56152.9453
Epoch 42/100
600/600 [===============

600/600 [==============================] - 0s 346us/step - loss: 83626.2284 - mae: 203.3477 - mse: 83626.2344
Epoch 2/100
600/600 [==============================] - 0s 37us/step - loss: 83493.4461 - mae: 203.0127 - mse: 83493.4453
Epoch 3/100
600/600 [==============================] - 0s 47us/step - loss: 83337.1099 - mae: 202.6398 - mse: 83337.1172
Epoch 4/100
600/600 [==============================] - 0s 40us/step - loss: 83141.0104 - mae: 202.1602 - mse: 83141.0078
Epoch 5/100
600/600 [==============================] - 0s 45us/step - loss: 82885.3076 - mae: 201.5272 - mse: 82885.3125
Epoch 6/100
600/600 [==============================] - 0s 48us/step - loss: 82557.2367 - mae: 200.6931 - mse: 82557.2266
Epoch 7/100
600/600 [==============================] - 0s 47us/step - loss: 82145.2366 - mae: 199.6808 - mse: 82145.2422
Epoch 8/100
600/600 [==============================] - 0s 45us/step - loss: 81674.1573 - mae: 198.4741 - mse: 81674.1562
Epoch 9/100
600/600 [======================

600/600 [==============================] - 0s 47us/step - loss: 43366.7755 - mae: 123.4778 - mse: 43366.7734
Epoch 69/100
600/600 [==============================] - 0s 55us/step - loss: 43287.5680 - mae: 123.6775 - mse: 43287.5703
Epoch 70/100
600/600 [==============================] - 0s 57us/step - loss: 43222.5524 - mae: 123.8406 - mse: 43222.5508
Epoch 71/100
600/600 [==============================] - 0s 52us/step - loss: 43159.4695 - mae: 124.0529 - mse: 43159.4727
Epoch 72/100
600/600 [==============================] - 0s 52us/step - loss: 43096.1289 - mae: 124.2905 - mse: 43096.1250
Epoch 73/100
600/600 [==============================] - 0s 43us/step - loss: 43026.9067 - mae: 124.3951 - mse: 43026.9062
Epoch 74/100
600/600 [==============================] - 0s 47us/step - loss: 42972.7618 - mae: 124.5320 - mse: 42972.7656
Epoch 75/100
600/600 [==============================] - 0s 45us/step - loss: 42911.2975 - mae: 124.6683 - mse: 42911.3008
Epoch 76/100
600/600 [===============

750/750 [==============================] - 0s 51us/step - loss: 55737.7762 - mae: 130.2397 - mse: 55737.7695
Epoch 36/100
750/750 [==============================] - 0s 44us/step - loss: 54871.2917 - mae: 128.4312 - mse: 54871.2891
Epoch 37/100
750/750 [==============================] - 0s 46us/step - loss: 54072.2356 - mae: 126.8180 - mse: 54072.2344
Epoch 38/100
750/750 [==============================] - 0s 44us/step - loss: 53308.5484 - mae: 125.3219 - mse: 53308.5430
Epoch 39/100
750/750 [==============================] - 0s 44us/step - loss: 52590.2095 - mae: 123.9994 - mse: 52590.2031
Epoch 40/100
750/750 [==============================] - 0s 41us/step - loss: 51880.5719 - mae: 122.8800 - mse: 51880.5742
Epoch 41/100
750/750 [==============================] - 0s 41us/step - loss: 51236.2767 - mae: 121.9479 - mse: 51236.2773
Epoch 42/100
750/750 [==============================] - 0s 45us/step - loss: 50666.9930 - mae: 121.2012 - mse: 50666.9922
Epoch 43/100
750/750 [===============

In [37]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[-41536.45578229 -32429.48374361 -61222.4536624  -34660.3325281
 -51258.86725425]
 
Mean and variance: 44221.52 (+/- 21470.61)
 
 
Model performance on testing data
Mean Absolute Error: 140.87187237426758
Root Mean Squared Error: 61803.214139767064
Time Taken =  46.578125


##### Medium depth Network

In [38]:
def build_model():
    model = Sequential()
    model.add(Dense(units=22, activation = 'relu', input_dim=22))
    model.add(Dense(units=22, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 0s 494us/step - loss: 86143.8211 - mae: 204.3756 - mse: 86143.8125
Epoch 2/100
600/600 [==============================] - 0s 37us/step - loss: 85948.7122 - mae: 203.9089 - mse: 85948.7188
Epoch 3/100
600/600 [==============================] - 0s 38us/step - loss: 85689.6510 - mae: 203.2637 - mse: 85689.6484
Epoch 4/100
600/600 [==============================] - 0s 45us/step - loss: 85306.2901 - mae: 202.3071 - mse: 85306.2891
Epoch 5/100
600/600 [==============================] - 0s 40us/step - loss: 84740.8529 - mae: 200.8789 - mse: 84740.8594
Epoch 6/100
600/600 [==============================] - 0s 40us/step - loss: 83899.0501 - mae: 198.7878 - mse: 83899.0625
Epoch 7/100
600/600 [==============================] - 0s 47us/step - loss: 82714.2490 - mae: 195.8471 - mse: 82714.2422
Epoch 8/100
600/600 [==============================] - 0s 37us/step - loss: 81112.7479 - mae: 191.7996 - mse: 81112.7422
Epoch 9/100
600/600 [==========

600/600 [==============================] - 0s 75us/step - loss: 37154.1696 - mae: 117.3677 - mse: 37154.1719
Epoch 69/100
600/600 [==============================] - 0s 75us/step - loss: 36943.7979 - mae: 117.2353 - mse: 36943.8008
Epoch 70/100
600/600 [==============================] - 0s 66us/step - loss: 36721.5549 - mae: 116.6792 - mse: 36721.5586
Epoch 71/100
600/600 [==============================] - 0s 66us/step - loss: 36470.2919 - mae: 116.3344 - mse: 36470.2930
Epoch 72/100
600/600 [==============================] - 0s 48us/step - loss: 36224.2459 - mae: 116.0610 - mse: 36224.2461
Epoch 73/100
600/600 [==============================] - 0s 53us/step - loss: 35990.4499 - mae: 115.4903 - mse: 35990.4492
Epoch 74/100
600/600 [==============================] - 0s 53us/step - loss: 35746.9488 - mae: 115.0453 - mse: 35746.9453
Epoch 75/100
600/600 [==============================] - 0s 60us/step - loss: 35520.6155 - mae: 115.1467 - mse: 35520.6172
Epoch 76/100
600/600 [===============

600/600 [==============================] - 0s 53us/step - loss: 47489.9531 - mae: 130.1230 - mse: 47489.9531
Epoch 36/100
600/600 [==============================] - 0s 50us/step - loss: 47338.8472 - mae: 129.9330 - mse: 47338.8516
Epoch 37/100
600/600 [==============================] - 0s 45us/step - loss: 47190.5262 - mae: 129.6803 - mse: 47190.5273
Epoch 38/100
600/600 [==============================] - 0s 61us/step - loss: 47066.5880 - mae: 129.8642 - mse: 47066.5820
Epoch 39/100
600/600 [==============================] - 0s 55us/step - loss: 46909.7028 - mae: 129.8322 - mse: 46909.7031
Epoch 40/100
600/600 [==============================] - 0s 51us/step - loss: 46759.1749 - mae: 129.6566 - mse: 46759.1758
Epoch 41/100
600/600 [==============================] - 0s 52us/step - loss: 46622.0120 - mae: 129.1065 - mse: 46622.0117
Epoch 42/100
600/600 [==============================] - 0s 58us/step - loss: 46474.1432 - mae: 129.0249 - mse: 46474.1484
Epoch 43/100
600/600 [===============

600/600 [==============================] - 0s 40us/step - loss: 78045.8362 - mae: 195.4926 - mse: 78045.8438
Epoch 3/100
600/600 [==============================] - 0s 58us/step - loss: 77947.4551 - mae: 195.2354 - mse: 77947.4531
Epoch 4/100
600/600 [==============================] - 0s 38us/step - loss: 77810.6182 - mae: 194.8733 - mse: 77810.6172
Epoch 5/100
600/600 [==============================] - 0s 62us/step - loss: 77595.8172 - mae: 194.3161 - mse: 77595.8125
Epoch 6/100
600/600 [==============================] - 0s 41us/step - loss: 77273.0161 - mae: 193.4781 - mse: 77273.0156
Epoch 7/100
600/600 [==============================] - 0s 33us/step - loss: 76807.1052 - mae: 192.2657 - mse: 76807.1094
Epoch 8/100
600/600 [==============================] - 0s 50us/step - loss: 76155.9984 - mae: 190.5510 - mse: 76156.0000
Epoch 9/100
600/600 [==============================] - 0s 70us/step - loss: 75266.5823 - mae: 188.2416 - mse: 75266.5859
Epoch 10/100
600/600 [======================

600/600 [==============================] - 0s 45us/step - loss: 35192.9460 - mae: 112.5298 - mse: 35192.9453
Epoch 70/100
600/600 [==============================] - 0s 50us/step - loss: 34961.8218 - mae: 111.8146 - mse: 34961.8242
Epoch 71/100
600/600 [==============================] - 0s 48us/step - loss: 34782.0775 - mae: 111.3778 - mse: 34782.0781
Epoch 72/100
600/600 [==============================] - 0s 52us/step - loss: 34596.9713 - mae: 110.9233 - mse: 34596.9688
Epoch 73/100
600/600 [==============================] - 0s 57us/step - loss: 34446.9883 - mae: 111.4315 - mse: 34446.9883
Epoch 74/100
600/600 [==============================] - 0s 50us/step - loss: 34204.4410 - mae: 110.5158 - mse: 34204.4414
Epoch 75/100
600/600 [==============================] - 0s 52us/step - loss: 34008.6684 - mae: 110.0513 - mse: 34008.6680
Epoch 76/100
600/600 [==============================] - 0s 52us/step - loss: 33805.9959 - mae: 109.6599 - mse: 33805.9961
Epoch 77/100
600/600 [===============

600/600 [==============================] - 0s 45us/step - loss: 45516.1577 - mae: 126.9154 - mse: 45516.1602
Epoch 37/100
600/600 [==============================] - 0s 50us/step - loss: 45365.3351 - mae: 126.9512 - mse: 45365.3320
Epoch 38/100
600/600 [==============================] - 0s 55us/step - loss: 45203.0597 - mae: 127.2283 - mse: 45203.0586
Epoch 39/100
600/600 [==============================] - 0s 60us/step - loss: 45064.9803 - mae: 125.9307 - mse: 45064.9844
Epoch 40/100
600/600 [==============================] - 0s 43us/step - loss: 44845.0131 - mae: 125.9252 - mse: 44845.0117
Epoch 41/100
600/600 [==============================] - 0s 52us/step - loss: 44663.3652 - mae: 126.1346 - mse: 44663.3594
Epoch 42/100
600/600 [==============================] - 0s 42us/step - loss: 44475.1734 - mae: 125.8249 - mse: 44475.1719
Epoch 43/100
600/600 [==============================] - 0s 53us/step - loss: 44308.1121 - mae: 125.6665 - mse: 44308.1133
Epoch 44/100
600/600 [===============

600/600 [==============================] - 0s 48us/step - loss: 83023.2352 - mae: 201.8495 - mse: 83023.2344
Epoch 4/100
600/600 [==============================] - 0s 55us/step - loss: 82636.6695 - mae: 200.8602 - mse: 82636.6641
Epoch 5/100
600/600 [==============================] - 0s 42us/step - loss: 81975.9018 - mae: 199.2447 - mse: 81975.8984
Epoch 6/100
600/600 [==============================] - 0s 47us/step - loss: 80990.9490 - mae: 196.6975 - mse: 80990.9453
Epoch 7/100
600/600 [==============================] - 0s 53us/step - loss: 79544.5318 - mae: 192.9431 - mse: 79544.5312
Epoch 8/100
600/600 [==============================] - 0s 40us/step - loss: 77465.8378 - mae: 187.8088 - mse: 77465.8359
Epoch 9/100
600/600 [==============================] - 0s 45us/step - loss: 74892.2264 - mae: 180.8766 - mse: 74892.2344
Epoch 10/100
600/600 [==============================] - 0s 50us/step - loss: 71723.0979 - mae: 172.4132 - mse: 71723.1094
Epoch 11/100
600/600 [=====================

600/600 [==============================] - 0s 80us/step - loss: 34933.7995 - mae: 115.9504 - mse: 34933.8008
Epoch 71/100
600/600 [==============================] - 0s 75us/step - loss: 34657.8537 - mae: 115.6816 - mse: 34657.8516
Epoch 72/100
600/600 [==============================] - 0s 70us/step - loss: 34416.2812 - mae: 115.1309 - mse: 34416.2852
Epoch 73/100
600/600 [==============================] - 0s 61us/step - loss: 34136.3715 - mae: 114.5787 - mse: 34136.3750
Epoch 74/100
600/600 [==============================] - 0s 68us/step - loss: 33895.6116 - mae: 114.7178 - mse: 33895.6133
Epoch 75/100
600/600 [==============================] - 0s 71us/step - loss: 33618.6079 - mae: 114.3808 - mse: 33618.6094
Epoch 76/100
600/600 [==============================] - 0s 67us/step - loss: 33378.4820 - mae: 114.0771 - mse: 33378.4844
Epoch 77/100
600/600 [==============================] - 0s 58us/step - loss: 33068.5338 - mae: 113.3402 - mse: 33068.5352
Epoch 78/100
600/600 [===============

750/750 [==============================] - 0s 64us/step - loss: 41901.7731 - mae: 123.5435 - mse: 41901.7734
Epoch 38/100
750/750 [==============================] - 0s 66us/step - loss: 41697.2096 - mae: 123.3323 - mse: 41697.2148
Epoch 39/100
750/750 [==============================] - 0s 59us/step - loss: 41472.5952 - mae: 122.9707 - mse: 41472.5938
Epoch 40/100
750/750 [==============================] - 0s 66us/step - loss: 41269.3595 - mae: 123.0672 - mse: 41269.3594
Epoch 41/100
750/750 [==============================] - 0s 53us/step - loss: 41051.9808 - mae: 122.3055 - mse: 41051.9805
Epoch 42/100
750/750 [==============================] - 0s 62us/step - loss: 40822.1702 - mae: 122.0865 - mse: 40822.1680
Epoch 43/100
750/750 [==============================] - 0s 53us/step - loss: 40604.1698 - mae: 122.1302 - mse: 40604.1719
Epoch 44/100
750/750 [==============================] - 0s 60us/step - loss: 40380.9407 - mae: 121.3874 - mse: 40380.9375
Epoch 45/100
750/750 [===============

In [39]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

Below are the scores for each model run
[-28565.39957291 -24083.02752101 -45652.81687063 -25565.40087296
 -37855.60690973]
 
Mean and variance: 32344.45 (+/- 16394.47)
 
 
Model performance on testing data
Mean Absolute Error: 108.11128180541992
Root Mean Squared Error: 41808.255924489065


##### Deep Network

In [40]:
def build_model():
    model = Sequential()
    model.add(Dense(units=11, activation = 'relu', input_dim=22))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 0s 660us/step - loss: 86327.0083 - mae: 204.8447 - mse: 86327.0078
Epoch 2/100
600/600 [==============================] - 0s 62us/step - loss: 86293.7312 - mae: 204.7630 - mse: 86293.7344
Epoch 3/100
600/600 [==============================] - 0s 55us/step - loss: 86251.3379 - mae: 204.6585 - mse: 86251.3438
Epoch 4/100
600/600 [==============================] - 0s 61us/step - loss: 86186.6818 - mae: 204.4871 - mse: 86186.6797
Epoch 5/100
600/600 [==============================] - 0s 53us/step - loss: 86063.7396 - mae: 204.1797 - mse: 86063.7344
Epoch 6/100
600/600 [==============================] - 0s 63us/step - loss: 85857.3141 - mae: 203.6518 - mse: 85857.3203
Epoch 7/100
600/600 [==============================] - 0s 47us/step - loss: 85516.0625 - mae: 202.7661 - mse: 85516.0703
Epoch 8/100
600/600 [==============================] - 0s 63us/step - loss: 84925.8935 - mae: 201.2662 - mse: 84925.8828
Epoch 9/100
600/600 [==========

600/600 [==============================] - 0s 52us/step - loss: 36053.6670 - mae: 116.7716 - mse: 36053.6680
Epoch 69/100
600/600 [==============================] - 0s 60us/step - loss: 35781.2469 - mae: 116.3723 - mse: 35781.2461
Epoch 70/100
600/600 [==============================] - 0s 55us/step - loss: 35441.9320 - mae: 115.1830 - mse: 35441.9336
Epoch 71/100
600/600 [==============================] - 0s 51us/step - loss: 35150.8938 - mae: 114.6777 - mse: 35150.8906
Epoch 72/100
600/600 [==============================] - 0s 51us/step - loss: 34898.0190 - mae: 114.9180 - mse: 34898.0195
Epoch 73/100
600/600 [==============================] - 0s 41us/step - loss: 34503.0789 - mae: 113.5288 - mse: 34503.0781
Epoch 74/100
600/600 [==============================] - 0s 47us/step - loss: 34196.9363 - mae: 112.4602 - mse: 34196.9336
Epoch 75/100
600/600 [==============================] - 0s 47us/step - loss: 33937.8187 - mae: 112.8537 - mse: 33937.8203
Epoch 76/100
600/600 [===============

600/600 [==============================] - 0s 50us/step - loss: 46175.6137 - mae: 128.6805 - mse: 46175.6133
Epoch 36/100
600/600 [==============================] - 0s 38us/step - loss: 46013.6649 - mae: 128.5733 - mse: 46013.6680
Epoch 37/100
600/600 [==============================] - 0s 43us/step - loss: 45855.7292 - mae: 128.4243 - mse: 45855.7266
Epoch 38/100
600/600 [==============================] - 0s 43us/step - loss: 45707.1106 - mae: 128.5693 - mse: 45707.1055
Epoch 39/100
600/600 [==============================] - 0s 45us/step - loss: 45553.1648 - mae: 128.4764 - mse: 45553.1680
Epoch 40/100
600/600 [==============================] - 0s 43us/step - loss: 45400.4520 - mae: 128.3483 - mse: 45400.4609
Epoch 41/100
600/600 [==============================] - 0s 48us/step - loss: 45244.8683 - mae: 128.5941 - mse: 45244.8672
Epoch 42/100
600/600 [==============================] - 0s 43us/step - loss: 45093.0848 - mae: 128.0452 - mse: 45093.0859
Epoch 43/100
600/600 [===============

600/600 [==============================] - 0s 43us/step - loss: 77950.5630 - mae: 195.2369 - mse: 77950.5703
Epoch 3/100
600/600 [==============================] - 0s 36us/step - loss: 77856.9758 - mae: 194.9929 - mse: 77856.9766
Epoch 4/100
600/600 [==============================] - 0s 43us/step - loss: 77714.1618 - mae: 194.6362 - mse: 77714.1641
Epoch 5/100
600/600 [==============================] - 0s 52us/step - loss: 77520.1396 - mae: 194.1287 - mse: 77520.1484
Epoch 6/100
600/600 [==============================] - 0s 47us/step - loss: 77239.6172 - mae: 193.4237 - mse: 77239.6250
Epoch 7/100
600/600 [==============================] - 0s 39us/step - loss: 76867.2092 - mae: 192.4442 - mse: 76867.2031
Epoch 8/100
600/600 [==============================] - 0s 38us/step - loss: 76351.7555 - mae: 191.0786 - mse: 76351.7500
Epoch 9/100
600/600 [==============================] - 0s 48us/step - loss: 75624.0031 - mae: 189.2323 - mse: 75623.9922
Epoch 10/100
600/600 [======================

600/600 [==============================] - 0s 38us/step - loss: 34388.9173 - mae: 110.6662 - mse: 34388.9219
Epoch 69/100
600/600 [==============================] - 0s 48us/step - loss: 34101.2782 - mae: 110.2782 - mse: 34101.2812
Epoch 70/100
600/600 [==============================] - 0s 45us/step - loss: 33823.2869 - mae: 109.2843 - mse: 33823.2891
Epoch 71/100
600/600 [==============================] - 0s 45us/step - loss: 33543.8845 - mae: 108.6801 - mse: 33543.8828
Epoch 72/100
600/600 [==============================] - 0s 52us/step - loss: 33226.0503 - mae: 108.7536 - mse: 33226.0508
Epoch 73/100
600/600 [==============================] - 0s 50us/step - loss: 32933.2418 - mae: 108.5593 - mse: 32933.2383
Epoch 74/100
600/600 [==============================] - 0s 53us/step - loss: 32623.7464 - mae: 107.8211 - mse: 32623.7441
Epoch 75/100
600/600 [==============================] - 0s 47us/step - loss: 32301.8743 - mae: 106.7956 - mse: 32301.8770
Epoch 76/100
600/600 [===============

600/600 [==============================] - 0s 50us/step - loss: 45875.1654 - mae: 127.1357 - mse: 45875.1680
Epoch 36/100
600/600 [==============================] - 0s 48us/step - loss: 45716.6814 - mae: 127.5134 - mse: 45716.6797
Epoch 37/100
600/600 [==============================] - 0s 47us/step - loss: 45494.4749 - mae: 128.1112 - mse: 45494.4727
Epoch 38/100
600/600 [==============================] - 0s 45us/step - loss: 45289.4227 - mae: 127.3615 - mse: 45289.4258
Epoch 39/100
600/600 [==============================] - 0s 50us/step - loss: 45128.7467 - mae: 126.8165 - mse: 45128.7461
Epoch 40/100
600/600 [==============================] - 0s 53us/step - loss: 44923.9367 - mae: 126.4089 - mse: 44923.9375
Epoch 41/100
600/600 [==============================] - 0s 59us/step - loss: 44735.6902 - mae: 126.8520 - mse: 44735.6914
Epoch 42/100
600/600 [==============================] - 0s 42us/step - loss: 44536.9253 - mae: 126.8538 - mse: 44536.9219
Epoch 43/100
600/600 [===============

600/600 [==============================] - 0s 37us/step - loss: 83556.9974 - mae: 203.1914 - mse: 83556.9922
Epoch 3/100
600/600 [==============================] - 0s 45us/step - loss: 83504.2034 - mae: 203.0643 - mse: 83504.2109
Epoch 4/100
600/600 [==============================] - 0s 42us/step - loss: 83426.1323 - mae: 202.8713 - mse: 83426.1328
Epoch 5/100
600/600 [==============================] - 0s 43us/step - loss: 83288.3857 - mae: 202.5312 - mse: 83288.3828
Epoch 6/100
600/600 [==============================] - 0s 57us/step - loss: 83036.7065 - mae: 201.9209 - mse: 83036.7109
Epoch 7/100
600/600 [==============================] - 0s 70us/step - loss: 82635.4333 - mae: 200.9397 - mse: 82635.4297
Epoch 8/100
600/600 [==============================] - 0s 43us/step - loss: 82014.6992 - mae: 199.3647 - mse: 82014.6875
Epoch 9/100
600/600 [==============================] - 0s 50us/step - loss: 81065.0125 - mae: 197.0285 - mse: 81065.0078
Epoch 10/100
600/600 [======================

600/600 [==============================] - 0s 40us/step - loss: 34110.1421 - mae: 115.0155 - mse: 34110.1406
Epoch 70/100
600/600 [==============================] - 0s 62us/step - loss: 33836.3831 - mae: 114.0521 - mse: 33836.3789
Epoch 71/100
600/600 [==============================] - 0s 53us/step - loss: 33596.0677 - mae: 114.4223 - mse: 33596.0625
Epoch 72/100
600/600 [==============================] - 0s 46us/step - loss: 33247.2163 - mae: 113.7416 - mse: 33247.2148
Epoch 73/100
600/600 [==============================] - 0s 42us/step - loss: 32965.1283 - mae: 112.5689 - mse: 32965.1289
Epoch 74/100
600/600 [==============================] - 0s 52us/step - loss: 32671.4678 - mae: 112.2303 - mse: 32671.4629
Epoch 75/100
600/600 [==============================] - 0s 59us/step - loss: 32375.5376 - mae: 111.7120 - mse: 32375.5371
Epoch 76/100
600/600 [==============================] - 0s 52us/step - loss: 32054.6598 - mae: 111.6360 - mse: 32054.6602
Epoch 77/100
600/600 [===============

750/750 [==============================] - 0s 49us/step - loss: 40765.4933 - mae: 123.0063 - mse: 40765.4922
Epoch 37/100
750/750 [==============================] - 0s 48us/step - loss: 40491.0617 - mae: 122.6298 - mse: 40491.0625
Epoch 38/100
750/750 [==============================] - 0s 52us/step - loss: 40215.6758 - mae: 121.8650 - mse: 40215.6758
Epoch 39/100
750/750 [==============================] - 0s 66us/step - loss: 39992.7375 - mae: 122.3258 - mse: 39992.7422
Epoch 40/100
750/750 [==============================] - 0s 86us/step - loss: 39668.3496 - mae: 120.3292 - mse: 39668.3516
Epoch 41/100
750/750 [==============================] - 0s 73us/step - loss: 39314.6056 - mae: 120.0727 - mse: 39314.6094
Epoch 42/100
750/750 [==============================] - 0s 66us/step - loss: 38990.9671 - mae: 121.2516 - mse: 38990.9688
Epoch 43/100
750/750 [==============================] - 0s 80us/step - loss: 38611.7329 - mae: 120.3363 - mse: 38611.7305
Epoch 44/100
750/750 [===============

In [41]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))

Below are the scores for each model run
[-25689.92047577 -16121.26387972 -40942.26717027 -24309.82918077
 -35726.95904963]
 
Mean and variance: 28558.05 (+/- 17566.09)
 
 
Model performance on testing data
Mean Absolute Error: 107.83027166259767
Root Mean Squared Error: 38047.33872083337
Time Taken =  61.15625
